In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
!ls '/content/data/'
!rm -r /content/data/*

common


In [3]:
import os
import pandas as pd
import pickle
import gdown
# data folder
url = "https://drive.google.com/drive/folders/1-9q8sUlBhj8L_Tc18mI5rLlWvxI6JW0-?usp=drive_link"
output_dir = '/content/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

gdown.download_folder(url, output=output_dir)

Retrieving folder contents


Retrieving folder 1-aVlnWRggV0rfhqBSUNnbKGARNfgL2T4 common
Processing file 16o7RxcM2edO_jedQX1OI4Oy2cQF3vXOQ __init__.py
Retrieving folder 1BxCK657k5wZNljexMoFuWYQvBI3tln_W __pycache__
Processing file 1-EAgBnmO206Vs9xLUOJzBp2841vgvHTd __init__.cpython-310.pyc
Processing file 1-E2xwNE1Z-ThHttjI5EaN2VR81a8xR1Z common.cpython-310.pyc
Processing file 1-DRT2KBjsR4k1NzPV3gtePzKXJKD_yLm pytorch.cpython-310.pyc
Processing file 1ZSfMTnVoRq-LpnVoqO8pNEnsEMc2tiL5 common.py
Processing file 19M3f8ePOVAgSDM1xxeRrwaUoVJGitU7n pytorch.py
Processing file 10vBmvv_4Ey4ZOp_cDH3y9aSsnXsTh8T3 data.parquet
Processing file 1AtA2fLk6HslJ8lIrKru33SSqROyUCNUc data1.parquet
Retrieving folder 1Jmq6qCOnqCxliC-RZOS9kNdk5CJJe4SU dataLoader
Processing file 1klB8-1EcGtf5WlDaCvw9yuOFpN2YXVPS __init__.py
Retrieving folder 1-KeVb3fzLZuCGbGexK-D-9u4btJX3HWX __pycache__
Processing file 1-tN81SdzC_LE4L3eK5ZGZ-pQb4k1gqIy __init__.cpython-310.pyc
Processing file 1-q3d0Oz1I2YUPVEJ_-N-U8_uNLK_PjDt MLM.cpython-310.pyc
Processing 

Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=16o7RxcM2edO_jedQX1OI4Oy2cQF3vXOQ
To: /content/data/common/__init__.py
0.00B [00:00, ?B/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-EAgBnmO206Vs9xLUOJzBp2841vgvHTd
From (redirected): https://drive.google.com/uc?id=1-EAgBnmO206Vs9xLUOJzBp2841vgvHTd&confirm=t&uuid=03340159-9725-4a24-858b-885e84bc55da
To: /content/data/common/__pycache__/__init__.cpython-310.pyc
100%|██████████| 164/164 [00:00<00:00, 166kB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-E2xwNE1Z-ThHttjI5EaN2VR81a8xR1Z
From (redirected): https://drive.google.com/uc?id=1-E2xwNE1Z-ThHttjI5EaN2VR81a8xR1Z&confirm=t&uuid=49f748da-86cd-4c39-bad6-f00ce5544eb6
To: /content/data/common/__pycache__/common.cpython-310.pyc
100%|██████████| 1.71k/1.71k [00:00<00:00, 1.70MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=

['/content/data/common/__init__.py',
 '/content/data/common/__pycache__/__init__.cpython-310.pyc',
 '/content/data/common/__pycache__/common.cpython-310.pyc',
 '/content/data/common/__pycache__/pytorch.cpython-310.pyc',
 '/content/data/common/common.py',
 '/content/data/common/pytorch.py',
 '/content/data/data.parquet',
 '/content/data/data1.parquet',
 '/content/data/dataLoader/__init__.py',
 '/content/data/dataLoader/__pycache__/__init__.cpython-310.pyc',
 '/content/data/dataLoader/__pycache__/MLM.cpython-310.pyc',
 '/content/data/dataLoader/__pycache__/utils.cpython-310.pyc',
 '/content/data/dataLoader/MLM_v0.py',
 '/content/data/dataLoader/MLM.py',
 '/content/data/dataLoader/NextXVisit.py',
 '/content/data/dataLoader/utils.py',
 '/content/data/figures/model.png',
 '/content/data/figures/results.png',
 '/content/data/Make_data_parquet_file.ipynb',
 '/content/data/MLM_UIUC1.ipynb',
 '/content/data/model/__init__.py',
 '/content/data/model/__pycache__/__init__.cpython-310.pyc',
 '/cont

In [4]:
import os

# Set the run directory path
run_directory = "/content/data/"

# Change the current working directory
os.chdir(run_directory)

# Verify the current working directory
print("Current working directory:", os.getcwd())

Current working directory: /content/data


In [5]:
!pip install pytorch-pretrained-bert

In [6]:
from common.common import create_folder
from common.pytorch import load_model
from model.utils import age_vocab
from common.common import load_obj
from dataLoader.MLM import MLMLoader
from torch.utils.data import DataLoader
import pandas as pd
from model.MLM import BertForMaskedLM
from model.optimiser import adam
import sklearn.metrics as skm
import numpy as np
import torch
import time
import torch.nn as nn
import os

from transformers import BertModel, BertConfig

In [7]:
class CustomBertConfig(BertConfig):
    def __init__(self, config):
        super(CustomBertConfig, self).__init__(
            vocab_size=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings = config.get('max_position_embedding'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')

class TrainConfig(object):
    def __init__(self, config):
        self.batch_size = config.get('batch_size')
        self.use_cuda = config.get('use_cuda')
        self.max_len_seq = config.get('max_len_seq')
        self.train_loader_workers = config.get('train_loader_workers')
        self.test_loader_workers = config.get('test_loader_workers')
        self.device = config.get('device')
        self.output_dir = config.get('output_dir')
        self.output_name = config.get('output_name')
        self.best_name = config.get('best_name')

In [8]:
file_config = {
    'vocab':'vocab_token_idx',  # vocabulary idx2token, token2idx
    'data': 'data.parquet',  # formated data
    'model_path': os.getcwd(), # where to save model
    'model_name': 'myMLM', # model name
    'file_name': 'Test',  # log path
}
full_path = os.path.join(file_config['model_path'], 'Test_t')
print(full_path)
#create_folder(file_config['model_path'])
create_folder(full_path)

/content/data/Test_t


In [9]:
global_params = {
    'max_seq_len': 64,
    'max_age': 110,
    'month': 12,
    'age_symbol': None,
    'min_visit': 1,
    'gradient_accumulation_steps': 1
}

optim_param = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

train_params = {
    'batch_size': 256,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'device': "cuda" if torch.cuda.is_available() else "cpu"
    #'device': "mps" if torch.backends.mps.is_available() else "cpu"
}

In [10]:
BertVocab = load_obj(file_config['vocab'])
ageVocab, _ = age_vocab(max_age=global_params['max_age'], mon=global_params['month'], symbol=global_params['age_symbol'])

In [11]:
data = pd.read_parquet(file_config['data'])
# remove patients with visits less than min visit
data['length'] = data['caliber_id'].apply(lambda x: len([i for i in range(len(x)) if x[i] == 'SEP']))
data = data[data['length'] >= global_params['min_visit']]
data = data.reset_index(drop=True)

In [12]:
data

,caliber_id,age,length
0,"[7455, 45829, V1259, 2724, SEP, 4239, 5119, 78...","[47, 47]",1
1,"[41071, 78551, 5781, 5849, 40391, 4280, 4592, ...","[87, 87]",1
2,"[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[71, 75]",1
3,"[41071, 4280, 4254, 42731, 9971, 4260, 41401, ...","[300, 304]",1
4,"[41401, 4111, 496, 4019, 3051, 53081, 60000, V...","[69, 69, 72]",2
...,...,...,...
7532,"[V5811, 1960, 2536, 1481, V1582, 4019, V1046, ...","[61, 61, 61]",2
7533,"[41401, 42823, 5185, 49322, 9994, 4280, 99527,...","[73, 74]",1
7534,"[25020, 5849, 42832, 4280, 2762, 5855, 40391, ...","[53, 54]",1
7535,"[45829, 4532, 2761, 5723, 4561, 45621, 5849, 7...","[54, 54]",1


In [13]:
Dset = MLMLoader(data, BertVocab['token2idx'], ageVocab, max_len=train_params['max_len_seq'], code='caliber_id')

In [ ]:
Dset = MLMLoader(data, BertVocab['token2idx'], ageVocab, max_len=train_params['max_len_seq'], code='caliber_id')
#print(BertVocab['token2idx'])
trainload = DataLoader(dataset=Dset, batch_size=train_params['batch_size'], shuffle=True, num_workers=3)

In [16]:
model_config = {
    'vocab_size': 100000, #len(BertVocab['token2idx'].keys())*8, # number of disease + symbols for word embedding
    'hidden_size': 288, # word embedding and seg embedding hidden size
    'seg_vocab_size': 2, # number of vocab for seg embedding
    'age_vocab_size': len(ageVocab.keys()), # number of vocab for age embedding
    'max_position_embedding': train_params['max_len_seq'], # maximum number of tokens
    'hidden_dropout_prob': 0.1, # dropout rate
    'num_hidden_layers': 6, # number of multi-head attention layers required
    'num_attention_heads': 12, # number of attention heads
    'attention_probs_dropout_prob': 0.1, # multi-head attention dropout rate
    'intermediate_size': 512, # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu', # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
    'initializer_range': 0.02, # parameter weight initializer range
}

In [17]:
print(model_config['vocab_size'])

100000


In [18]:
conf = CustomBertConfig(model_config)
model = BertForMaskedLM(conf)

In [19]:
model = model.to(train_params['device'])
optim = adam(params=list(model.named_parameters()), config=optim_param)

In [20]:
def cal_acc(label, pred):
    logs = nn.LogSoftmax()
    label=label.cpu().numpy()
    ind = np.where(label!=-1)[0]
    truepred = pred.detach().cpu().numpy()
    truepred = truepred[ind]
    truelabel = label[ind]
    truepred = logs(torch.tensor(truepred))
    outs = [np.argmax(pred_x) for pred_x in truepred.numpy()]
    precision = skm.precision_score(truelabel, outs, average='micro')
    return precision

In [21]:
def train(e, loader):
    tr_loss = 0
    temp_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    cnt= 0
    start = time.time()

    for step, batch in enumerate(loader):
        cnt +=1
        batch = tuple(t.to(train_params['device']) for t in batch)
        age_ids, input_ids, posi_ids, segment_ids, attMask, masked_label = batch

        loss, pred, label = model(input_ids, age_ids, segment_ids, posi_ids,attention_mask=attMask, masked_lm_labels=masked_label)

        if global_params['gradient_accumulation_steps'] >1:
            loss = loss/global_params['gradient_accumulation_steps']
        loss.backward()

        temp_loss += loss.item()
        tr_loss += loss.item()

        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1

        if step % 200==0:
            print("epoch: {}\t| cnt: {}\t|Loss: {}\t| precision: {:.4f}\t| time: {:.2f}".format(e, cnt, temp_loss/2000, cal_acc(label, pred), time.time()-start))
            temp_loss = 0
            start = time.time()

        if (step + 1) % global_params['gradient_accumulation_steps'] == 0:
            optim.step()
            optim.zero_grad()

    print("** ** * Saving fine - tuned model ** ** * ")
    model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
    create_folder(file_config['model_path'])
    output_model_file = os.path.join(file_config['model_path'], file_config['model_name'])

    torch.save(model_to_save.state_dict(), output_model_file)

    cost = time.time() - start
    return tr_loss, cost

In [22]:
data_len = len(Dset)

In [23]:
f = open(os.path.join(file_config['model_path'], file_config['file_name']), "w")
f.write('{}\t{}\t{}\n'.format('epoch', 'loss', 'time'))
for e in range(50):
    loss, time_cost = train(e, trainload)
    loss = loss/data_len
    f.write('{}\t{}\t{}\n'.format(e, loss, time_cost))
f.close()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


epoch: 0	| cnt: 1	|Loss: 0.0058879051208496095	| precision: 0.0000	| time: 6.99


/usr/local/lib/python3.10/dist-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1630.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


** ** * Saving fine - tuned model ** ** * 


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


...

epoch: 47	| cnt: 1	|Loss: 8.369119605049491e-07	| precision: 1.0000	| time: 5.89
** ** * Saving fine - tuned model ** ** * 


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


epoch: 48	| cnt: 1	|Loss: 8.101253770291805e-07	| precision: 1.0000	| time: 5.94
** ** * Saving fine - tuned model ** ** * 


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


epoch: 49	| cnt: 1	|Loss: 7.847143569961191e-07	| precision: 1.0000	| time: 5.88
** ** * Saving fine - tuned model ** ** * 
